# Preparing Data
Sources:
* [World Happiness Report](https://worldhappiness.report/ed/2021/#appendices-and-data)
* [UN Statistics Division](https://unstats.un.org/unsd/methodology/m49/overview/)

In [1]:
import pandas as pd, numpy as np, re, ssl, requests

In [2]:
#This grabs the file from the World Happiness Report's aws s3 bucket.
response = requests.get("https://happiness-report.s3.amazonaws.com/2021/DataPanelWHR2021C2.xls")

#This creates a new file for the world happiness report to save it in the repository.
with open('DataPanelWHR2021C2.xls', 'wb') as output:
    output.write(response.content)
    output.close()

In [3]:
happy = pd.read_excel("DataPanelWHR2021C2.xls", engine="xlrd")
happy.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.723590,7.370100,0.450662,50.799999,0.718114,0.167640,0.881686,0.517637,0.258195
1,Afghanistan,2009,4.401778,7.539972,0.552308,51.200001,0.678896,0.190099,0.850035,0.583926,0.237092
2,Afghanistan,2010,4.758381,7.646709,0.539075,51.599998,0.600127,0.120590,0.706766,0.618265,0.275324
3,Afghanistan,2011,3.831719,7.619532,0.521104,51.919998,0.495901,0.162427,0.731109,0.611387,0.267175
4,Afghanistan,2012,3.782938,7.705479,0.520637,52.240002,0.530935,0.236032,0.775620,0.710385,0.267919


In [4]:
happy_mapper = {"Country name":"Country"}
happy.rename(columns=happy_mapper, inplace = True)
happy.columns

Index(['Country', 'year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Positive affect', 'Negative affect'],
      dtype='object')

In [5]:
happy_sort = happy.sort_values(by=['year'], ascending = False)
happy_sort.head()

,Country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1948,Zimbabwe,2020,3.159802,7.828757,0.717243,56.799999,0.643303,-0.008696,0.788523,0.702573,0.345736
174,Benin,2020,4.407746,8.102292,0.506636,55.099998,0.783115,-0.083489,0.531884,0.608585,0.304512
1835,United Kingdom,2020,6.798177,10.625811,0.929353,72.699997,0.884624,0.202508,0.490204,0.758164,0.224655
1394,Philippines,2020,5.079585,9.061443,0.781140,62.099998,0.932042,-0.115543,0.744284,0.803562,0.326889
785,Iraq,2020,4.785165,9.167186,0.707847,61.400002,0.700215,-0.020748,0.849109,0.644464,0.531539


In [6]:
happy_dedup = happy_sort.drop_duplicates(subset=['Country'])
happy_dedup.head()

,Country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1948,Zimbabwe,2020,3.159802,7.828757,0.717243,56.799999,0.643303,-0.008696,0.788523,0.702573,0.345736
174,Benin,2020,4.407746,8.102292,0.506636,55.099998,0.783115,-0.083489,0.531884,0.608585,0.304512
1835,United Kingdom,2020,6.798177,10.625811,0.929353,72.699997,0.884624,0.202508,0.490204,0.758164,0.224655
1394,Philippines,2020,5.079585,9.061443,0.781140,62.099998,0.932042,-0.115543,0.744284,0.803562,0.326889
785,Iraq,2020,4.785165,9.167186,0.707847,61.400002,0.700215,-0.020748,0.849109,0.644464,0.531539


In [7]:
len(happy_sort)

1949

In [8]:
len(happy_dedup)

166

In [9]:
#Note, the UN csv is delimited by semi-colons: ";".
m49 = pd.read_csv("UNSD-M49-countries-7-22.csv", sep=";")
mapper = {"Country or Area":"Country","Region Name":"Continent"}
m49.rename(columns=mapper, inplace=True)
m49.head()

,Global Code,Global Name,Region Code,Continent,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,NaN
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,NaN
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,NaN


In [10]:
len(m49)

249

## Identifying discrepencies and aligning on what to call countries

Knowing that the two data sources come from places with very different ideologies I know I will have to make the final decision as to what names end up in my data set. On top of that, certain places that people understand as countries (Ex. `Taiwan`) are not recognized by the UN at all. What I did was fairly straight forward:
1. I checked the overlap and identified how many countries appeared in each list
2. I then created a list of the ones that did not appear
3. I then determined how to handle discrepencies.
    1. If a place needed to be added to the UN list, then it gets added (Ex. Taiwan)
    2. If a place needed to be changed in either or both lists, then I update them both 
    2. I decided against removing territories, regions, etc. to make the list as inclusive as possible given the data at hand and my desire to make it broadly applicable.
4. Then I merge the two data frames on the UN Data Set, as it's larger
5. I export it to `combined-countries.csv`

In [11]:
c1 = list(happy_dedup["Country"])
c2 = list(m49["Country"])
c3 = [(country, country in c2) for country in c1]
c4 = [(country, country in c1) for country in c2]

In [12]:
print(len(c3))
c3[:10]

166


[('Zimbabwe', True),
 ('Benin', True),
 ('United Kingdom', False),
 ('Philippines', True),
 ('Iraq', True),
 ('Belgium', True),
 ('Iran', False),
 ('Poland', True),
 ('Portugal', True),
 ('India', True)]

In [13]:
print(len(c4))
c4[:10]

249


[('Algeria', True),
 ('Egypt', True),
 ('Libya', True),
 ('Morocco', True),
 ('Sudan', True),
 ('Tunisia', True),
 ('Western Sahara', False),
 ('British Indian Ocean Territory', False),
 ('Burundi', True),
 ('Comoros', True)]

In [14]:
num = 0
happy_missing = []
for pair in c3:
    if pair[1]:
        num +=1
    else:
        happy_missing.append(pair[0])
print(num)
print(len(happy_missing))
print(happy_missing)

142
24
['United Kingdom', 'Iran', 'Hong Kong S.A.R. of China', 'Bolivia', 'Russia', 'Laos', 'Kosovo', 'United States', 'Ivory Coast', 'Czech Republic', 'South Korea', 'Taiwan Province of China', 'Tanzania', 'Turkey', 'Venezuela', 'Moldova', 'Swaziland', 'Palestinian Territories', 'North Cyprus', 'Congo (Brazzaville)', 'Vietnam', 'Congo (Kinshasa)', 'Syria', 'Somaliland region']


In [15]:
num = 0
missing = []
for pair in c4:
    if pair[1]:
        num +=1
    else:
        missing.append(pair[0])
print(num)
print(len(missing))
print(missing)

142
107
['Western Sahara', 'British Indian Ocean Territory', 'Eritrea', 'French Southern Territories', 'Mayotte', 'Réunion', 'Seychelles', 'United Republic of Tanzania', 'Congo', 'Democratic Republic of the Congo', 'Equatorial Guinea', 'Sao Tome and Principe', 'Eswatini', 'Cabo Verde', 'Côte d’Ivoire', 'Guinea-Bissau', 'Saint Helena', 'Anguilla', 'Antigua and Barbuda', 'Aruba', 'Bahamas', 'Barbados', 'Bonaire, Sint Eustatius and Saba', 'British Virgin Islands', 'Cayman Islands', 'Curaçao', 'Dominica', 'Grenada', 'Guadeloupe', 'Martinique', 'Montserrat', 'Puerto Rico', 'Saint Barthélemy', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French Part)', 'Saint Vincent and the Grenadines', 'Sint Maarten (Dutch part)', 'Turks and Caicos Islands', 'United States Virgin Islands', 'Bolivia (Plurinational State of)', 'Bouvet Island', 'Falkland Islands (Malvinas)', 'French Guiana', 'South Georgia and the South Sandwich Islands', 'Venezuela (Bolivarian Republic of)', 'Bermuda', 'Greenland',

In [16]:
#Copies the dataframe to allow us to modify one and check on the other.
m49_clean = m49.copy()
m49_simple = m49.copy()

In [17]:
m49_targets = ["Viet Nam", "United States of America", "Russian Federation",  "Lao People's Democratic Republic", "United Kingdom of Great Britain and Northern Ireland", "Türkiye", "Czechia", "United Republic of Tanzania", "Côte d’Ivoire", "China, Macao Special Administrative Region", "China, Hong Kong Special Administrative Region", "Iran (Islamic Republic of)", "State of Palestine", "Syrian Arab Republic", "Democratic People's Republic of Korea", "Republic of Korea", "Micronesia (Federated States of)", "Bolivia (Plurinational State of)", "Republic of Moldova", "Venezuela (Bolivarian Republic of)"]
m49_updates = ["Vietnam", "United States", "Russia", "Laos", "United Kingdom", "Turkey", "Czech Republic", "Tanzania", "Ivory Coast", "Macao Special Administrative Region", "Hong Kong Special Administrative Region", "Iran", "Palestine", "Syria", "North Korea", "South Korea", "Micronesia", "Bolivia", "Moldova", "Venezuela"]

In [18]:
m49_simple.replace(m49_targets,m49_updates,inplace=True)

In [19]:
def create_lookup(terms,changes,column,df):
    '''This is a simple helper function to take a list of terms and a list of changes.
    They have to be of the same length and in the same order.
    It turns them into 2 dictionaries with the following key/pairs:
    targets: Key = term: Value = index
    updates: Key = index: Value = term'''
    values = []
    for term in terms:
        current = df.index[df[column] == term].tolist()
        values.append(current[0])
    target = dict(zip(terms,values))
    updates = dict(zip(values,changes))
    return target, updates   

In [20]:
s,q = create_lookup(m49_targets,m49_updates,"Country",m49)

In [21]:
#Checking that my code functions as intendend.
for k in s:
    print(k + " --> " + str(q[s[k]]))

Viet Nam --> Vietnam
United States of America --> United States
Russian Federation --> Russia
Lao People's Democratic Republic --> Laos
United Kingdom of Great Britain and Northern Ireland --> United Kingdom
Türkiye --> Turkey
Czechia --> Czech Republic
United Republic of Tanzania --> Tanzania
Côte d’Ivoire --> Ivory Coast
China, Macao Special Administrative Region --> Macao Special Administrative Region
China, Hong Kong Special Administrative Region --> Hong Kong Special Administrative Region
Iran (Islamic Republic of) --> Iran
State of Palestine --> Palestine
Syrian Arab Republic --> Syria
Democratic People's Republic of Korea --> North Korea
Republic of Korea --> South Korea
Micronesia (Federated States of) --> Micronesia
Bolivia (Plurinational State of) --> Bolivia
Republic of Moldova --> Moldova
Venezuela (Bolivarian Republic of) --> Venezuela


In [22]:
#Gets the index of the value from m49 to put it in m49_clean
x = m49.index[m49['Country'] == "Viet Nam"].tolist()

In [23]:
m49_clean.loc[x[0],'Country'] = "Vietnam"
y = m49_clean.index[m49_clean['Country'] == "Vietnam"].tolist()
y

[140]

In [24]:
def update_names(targets,changes,column,target_df):
    '''This helper function uses the dictionaries created in create_lookup to 
    update the dataframe at scale.'''
    for t in targets:
        target_df.at[targets[t],column] = changes[targets[t]]
    return target_df

In [25]:
m49_clean = update_names(s,q,'Country',m49_clean)

In [26]:
def check_presence(df1,df2,column,first=True):
    c1 = list(df1[column])
    c2 = list(df2[column])
    c3 = [(entry, entry in c2) for entry in c1]
    c4 = [(entry, entry in c1) for entry in c2]
    num = 0
    missing = []
    if first:
        for pair in c3:
            if pair[1]:
                num +=1
            else:
                missing.append(pair[0])
    else:
        for pair in c4:
            if pair[1]:
                num +=1
            else:
                missing.append(pair[0])
    print(num)
    print(len(missing))
    return missing

In [27]:
check_presence(happy_dedup,m49_clean,"Country")

157
9


['Hong Kong S.A.R. of China',
 'Kosovo',
 'Taiwan Province of China',
 'Swaziland',
 'Palestinian Territories',
 'North Cyprus',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Somaliland region']

In [28]:
m49_clean.equals(m49_simple)

True

In [29]:
happy_updates = ["Hong Kong S.A.R. of China", "Congo (Brazzaville)", "Congo (Kinshasa)", "Palestinian Territories", "Taiwan Province of China", "Swaziland"]
happy_changes = ["Hong Kong Special Administrative Region", "Congo","Democratic Republic of the Congo", "Palestine", "Taiwan", "Eswatini"]

In [30]:
print(len(happy_updates)==len(happy_changes))

True


In [31]:
happy_clean = happy_dedup.copy()

In [32]:
h_targets,h_changes = create_lookup(happy_updates,happy_changes,'Country',happy_dedup)
happy_clean = update_names(h_targets,h_changes,'Country',happy_clean)

In [33]:
#Checking that my code functions as intendend.
for k in h_targets:
    print(k + " --> " + str(h_changes[h_targets[k]]))

Hong Kong S.A.R. of China --> Hong Kong Special Administrative Region
Congo (Brazzaville) --> Congo
Congo (Kinshasa) --> Democratic Republic of the Congo
Palestinian Territories --> Palestine
Taiwan Province of China --> Taiwan
Swaziland --> Eswatini


In [34]:
missing = check_presence(happy_clean,m49_clean,"Country")
missing

162
4


['Kosovo', 'Taiwan', 'North Cyprus', 'Somaliland region']

In [35]:
m49.columns

Index(['Global Code', 'Global Name', 'Region Code', 'Continent',
       'Sub-region Code', 'Sub-region Name', 'Intermediate Region Code',
       'Intermediate Region Name', 'Country', 'M49 Code', 'ISO-alpha2 Code',
       'ISO-alpha3 Code', 'Least Developed Countries (LDC)',
       'Land Locked Developing Countries (LLDC)',
       'Small Island Developing States (SIDS)'],
      dtype='object')

In [36]:
missing = ['Kosovo', 'Taiwan', 'North Cyprus', 'Somaliland region']
m_continents = ['Europe', 'Asia', 'Europe','Africa']
idx = 0
for x in missing:
    current = pd.Series([None,None,None,m_continents[idx],None,None,None,None,x,None,None,None,None,None,None], index=m49.columns)
    m49_clean = m49_clean.append(current, ignore_index=True)
    idx += 1

/var/folders/lx/xt9qnk8569n7xy_d7knh3npr0000gp/T/ipykernel_82206/3391363385.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m49_clean = m49_clean.append(current, ignore_index=True)
/var/folders/lx/xt9qnk8569n7xy_d7knh3npr0000gp/T/ipykernel_82206/3391363385.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m49_clean = m49_clean.append(current, ignore_index=True)
/var/folders/lx/xt9qnk8569n7xy_d7knh3npr0000gp/T/ipykernel_82206/3391363385.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m49_clean = m49_clean.append(current, ignore_index=True)
/var/folders/lx/xt9qnk8569n7xy_d7knh3npr0000gp/T/ipykernel_82206/3391363385.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

In [51]:
#I choose to remove the parenthetical elements because I believe people are less likely to include them in natural language, and they will impact the generation of counterfactuals.
m49_clean.replace("\(.*\)",value="",regex=True,inplace=True)

In [44]:
print(m49_clean.Country.unique())

['Algeria' 'Egypt' 'Libya' 'Morocco' 'Sudan' 'Tunisia' 'Western Sahara'
 'British Indian Ocean Territory' 'Burundi' 'Comoros' 'Djibouti' 'Eritrea'
 'Ethiopia' 'French Southern Territories' 'Kenya' 'Madagascar' 'Malawi'
 'Mauritius' 'Mayotte' 'Mozambique' 'Réunion' 'Rwanda' 'Seychelles'
 'Somalia' 'South Sudan' 'Uganda' 'Tanzania' 'Zambia' 'Zimbabwe' 'Angola'
 'Cameroon' 'Central African Republic' 'Chad' 'Congo'
 'Democratic Republic of the Congo' 'Equatorial Guinea' 'Gabon'
 'Sao Tome and Principe' 'Botswana' 'Eswatini' 'Lesotho' 'Namibia'
 'South Africa' 'Benin' 'Burkina Faso' 'Cabo Verde' 'Ivory Coast' 'Gambia'
 'Ghana' 'Guinea' 'Guinea-Bissau' 'Liberia' 'Mali' 'Mauritania' 'Niger'
 'Nigeria' 'Saint Helena' 'Senegal' 'Sierra Leone' 'Togo' 'Anguilla'
 'Antigua and Barbuda' 'Aruba' 'Bahamas' 'Barbados'
 'Bonaire, Sint Eustatius and Saba' 'British Virgin Islands'
 'Cayman Islands' 'Cuba' 'Curaçao' 'Dominica' 'Dominican Republic'
 'Grenada' 'Guadeloupe' 'Haiti' 'Jamaica' 'Martinique' 'Mo

In [45]:
combo = m49_clean.set_index('Country').join(happy_clean.set_index('Country'), on='Country', how='left')

In [46]:
combo = combo.reset_index()

In [47]:
combo = combo.drop_duplicates(subset=['Country'])

In [48]:
len(combo)

253

In [49]:
combo.head()

,Country,Global Code,Global Name,Region Code,Continent,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,M49 Code,...,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Algeria,1.0,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,12.0,...,2019.0,4.744627,9.336946,0.803259,66.099998,0.385083,0.005087,0.740609,0.584944,0.215198
1,Egypt,1.0,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,818.0,...,2020.0,4.472397,9.382727,0.672725,62.299999,0.769550,-0.112342,NaN,0.598909,0.442034
2,Libya,1.0,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,434.0,...,2019.0,5.330222,9.627350,0.826719,62.299999,0.761964,-0.072673,0.686413,0.708741,0.400737
3,Morocco,1.0,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,504.0,...,2020.0,4.802618,8.870917,0.552520,66.500000,0.818995,-0.228578,0.802740,0.587182,0.256431
4,Sudan,1.0,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,729.0,...,2014.0,4.138673,8.317068,0.810616,55.119999,0.390096,-0.063395,0.793785,0.540845,0.302725


In [50]:
combo.to_csv("combined-countries.csv", index=False)